# Setup

In [1]:
%%capture
!pip install --upgrade optuna
!pip install --upgrade lightgbm

In [2]:
import os
import gc
import time
import warnings

gc.enable()
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)
np.set_printoptions(precision=4)

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner

import lightgbm as lgb
from lightgbm import LGBMClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, average_precision_score

In [3]:
#remove cell to run future versions
assert optuna.__version__ == '3.0.3', f'Change in Optuna version. Original notebook version: 3.0.2'
assert lgb.__version__ == '3.3.3', f'Change in LightGBM version. Original notebook version: 3.3.3'

In [4]:
SEED = 2311
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)

In [5]:
DATA_URL = 'https://raw.githubusercontent.com/stiwari-ds/data-science-competitions/main/analytics_vidhya/dataverse_hack/data'

train = pd.read_csv(f'{DATA_URL}/processed/train.csv') #processed dataset from notebook 00
test = pd.read_csv(f'{DATA_URL}/processed/test.csv')
sample_sub = pd.read_csv(f'{DATA_URL}/raw/sample_submission.csv')

In [6]:
TARGET = 'is_claim'

In [7]:
features = list(test.columns)

num_features = ['policy_tenure', 'age_of_car', 'age_of_policyholder', 
                'population_density', 'airbags', 'displacement', 'turning_radius',
                'length', 'width','height', 'gross_weight', 'ncap_rating']

cat_features = [f for f in features if f not in num_features]

# Baseline

In [8]:
def get_best_threshold(y_true, pred_probs):
    candidate_thresholds = np.arange(0, 1, 0.01)
    candidate_scores = [f1_score(y_true, (pred_probs >= t).astype('int')) 
                        for t in candidate_thresholds]
    best_threshold = candidate_thresholds[np.argmax(candidate_scores)]
    return best_threshold

In [11]:
%%time
scores_f1 = []
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
X, y = train[features], train[TARGET]
feature_name = list(X.columns)
categorical_feature = [f for f in feature_name if f in cat_features]
for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
    X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.loc[val_idx], y.iloc[val_idx]

    model = LGBMClassifier(
        objective='binary',
        boosting_type='goss',
        device_type='cpu',
        random_state=SEED
    ) 
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        early_stopping_rounds=100,
        eval_metric='average_precision',
        feature_name=feature_name,
        categorical_feature=categorical_feature,
        verbose=0
    )
    val_probs = model.predict_proba(X_val)[:, 1]
    best_threshold = get_best_threshold(y_val, val_probs)
    val_preds = (val_probs >= best_threshold).astype('int')

    score = f1_score(y_val, val_preds)
    scores_f1.append(score)
    print(f'Fold #{fold}: ({model.best_iteration_} rounds) F1-score = {score:.5f}')
    _ = gc.collect()

print(f'\nAvg F1-score = {np.mean(scores_f1):.5f} +/- {np.std(scores_f1):.5f}\n')

Fold #0: (6 rounds) F1-score = 0.17071
Fold #1: (8 rounds) F1-score = 0.17465
Fold #2: (19 rounds) F1-score = 0.17150
Fold #3: (7 rounds) F1-score = 0.17308
Fold #4: (11 rounds) F1-score = 0.16835

Avg F1-score = 0.17166 +/- 0.00214

CPU times: user 15.4 s, sys: 137 ms, total: 15.5 s
Wall time: 10.1 s


# Hyperparameter tuning

In [23]:
def objective(trial, data, model, proba):

    scores = []
    X, y = data
    feature_name = list(X.columns)
    categorical_feature = [f for f in feature_name if f in cat_features]

    param_grid = {
        'learning_rate': trial.suggest_float('learning_rate', 0.025, 0.3, step=0.025),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 200, step=0.1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 200, step=0.1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 2000, step=5),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 0, 1000, step=2),
        'min_split_gain': trial.suggest_float('min_split_gain', 0, 10, step=0.01),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.95, step=0.05),
        'top_rate': trial.suggest_float('top_rate', 0.1, 0.5, step=0.05),
        'other_rate': trial.suggest_float('other_rate', 0.05, 0.5, step=0.05),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 15, step=0.05),
        'max_cat_to_onehot': trial.suggest_categorical('max_cat_to_onehot', [3, 6, 11, 22])
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.loc[val_idx], y.iloc[val_idx]
        
        model.set_params(**param_grid)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_metric='average_precision',
            early_stopping_rounds=100,
            feature_name=feature_name,
            categorical_feature=categorical_feature,
            verbose=False
        )
        if proba:
            val_preds = model.predict_proba(X_val)[:, 1]
            scores.append(average_precision_score(y_val, val_preds))
        else:
            val_preds = model.predict(X_val)
            scores.append(f1_score(y_val, val_preds))

    return np.mean(scores)

In [24]:
def tune_params(data, model, proba, n_trials=10, direction='maximize'):
    study = optuna.create_study(
        sampler=TPESampler(seed=SEED),
        pruner=HyperbandPruner(),
        direction=direction
    )
    study.optimize(
        func=lambda trial: objective(trial, data, model, proba),
        n_trials=n_trials,
        gc_after_trial=True
    )
    return study

# Cross-validation

In [25]:
def cross_validate_predict(data, model, proba, n_splits=5):
    oof_preds = {}  #out-of-fold predictions on train set
    test_preds = {} #predictions on test set for each fold
    scores = [] #scores on validation set

    X, y, X_test = data
    feature_name = list(X.columns)
    categorical_feature = [f for f in feature_name if f in cat_features]
       
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.loc[val_idx], y.iloc[val_idx]
        
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_metric='average_precision',
            early_stopping_rounds=100,
            feature_name=feature_name,
            categorical_feature=categorical_feature,
            verbose=False
        )

        if proba:
            val_probs = model.predict_proba(X_val)[:, 1]
            test_probs = model.predict_proba(X_test)[:, 1]
            best_threshold = get_best_threshold(y_val, val_probs)
            val_preds = (val_probs >= best_threshold).astype('int')
            test_preds_fold = (test_probs >= best_threshold).astype('int')
        else:
            val_preds = model.predict(X_val)
            test_preds_fold = model.predict(X_test)
        
        oof_preds.update(dict(zip(val_idx, val_preds)))
        test_preds[f'fold{fold}'] = test_preds_fold

        score = f1_score(y_val, val_preds)
        scores.append(score)
        print(f'Fold #{fold}: F1 = {score:.5f}')
        _ = gc.collect()
    print(f'Avg. F1 = {np.mean(scores):.5f} +/- {np.std(scores):.5f}')
    
    oof_preds = pd.Series(oof_preds).sort_index()
    test_preds = pd.DataFrame.from_dict(test_preds)
    test_preds['mode'] = test_preds.mode(axis=1)[0].astype('int')

    return oof_preds, test_preds

In [26]:
def run_experiment(data, proba, n_trials=10):
    
    X, y, X_test = data

    base_params = {
        'objective': 'binary',
        'n_estimators': 10000,
        'boosting_type': 'goss',
        'extra_trees': True,
        'verbosity': -1,
        'device_type': 'cpu',
        'random_state': SEED
    }
    
    model = LGBMClassifier(**base_params)
    
    print(f'----------Hyperparameter tuning----------')
    start = time.time()
    study = tune_params(
        data=(X, y),
        model=model,
        proba=proba,
        n_trials=n_trials, 
        direction='maximize' #metric: f1-score/avg_precision -> higher is better
    )
    end = time.time()
    print(f'Best trial: {study.best_trial.number} -> Best value: {study.best_value:.5f}')
    print(f'Best hyperparameters:')
    for k, v in study.best_params.items():
        print(f'{k:15} - {v}')
    print(f'[Time taken: {end - start:.2f}s]\n')
    
    print(f'-----Cross-validation and prediction-----')
    start = time.time()
    model.set_params(**study.best_params)
    oof_preds, test_preds = cross_validate_predict(data, model, proba)
    end = time.time()
    print(f'[Time taken: {end - start:.2f}s]\n')

    return oof_preds, test_preds

**Trial runs**

In [27]:
optuna.logging.set_verbosity(optuna.logging.INFO)

In [28]:
%%time
op, tp = run_experiment(
    data=(train[features], train[TARGET], test[features]),
    proba=False,
    n_trials=3
)

[I 2022-11-13 14:12:05,682] A new study created in memory with name: no-name-cbd0a840-fad6-4a34-9008-977a4c50356c


----------Hyperparameter tuning----------


[I 2022-11-13 14:12:09,905] Trial 0 finished with value: 0.0 and parameters: {'learning_rate': 0.1, 'reg_alpha': 169.20000000000002, 'reg_lambda': 43.7, 'num_leaves': 315, 'max_depth': 3, 'min_child_samples': 708, 'min_split_gain': 0.14, 'colsample_bytree': 0.55, 'top_rate': 0.4, 'other_rate': 0.3, 'scale_pos_weight': 1.75, 'max_cat_to_onehot': 22}. Best is trial 0 with value: 0.0.
[I 2022-11-13 14:12:13,373] Trial 1 finished with value: 0.0 and parameters: {'learning_rate': 0.17500000000000002, 'reg_alpha': 23.700000000000003, 'reg_lambda': 133.9, 'num_leaves': 720, 'max_depth': 11, 'min_child_samples': 548, 'min_split_gain': 7.69, 'colsample_bytree': 0.7, 'top_rate': 0.2, 'other_rate': 0.05, 'scale_pos_weight': 11.600000000000001, 'max_cat_to_onehot': 11}. Best is trial 0 with value: 0.0.
[I 2022-11-13 14:12:16,803] Trial 2 finished with value: 0.0 and parameters: {'learning_rate': 0.225, 'reg_alpha': 183.20000000000002, 'reg_lambda': 73.2, 'num_leaves': 860, 'max_depth': 15, 'min_ch

Best trial: 0 -> Best value: 0.00000
Best hyperparameters:
learning_rate   - 0.1
reg_alpha       - 169.20000000000002
reg_lambda      - 43.7
num_leaves      - 315
max_depth       - 3
min_child_samples - 708
min_split_gain  - 0.14
colsample_bytree - 0.55
top_rate        - 0.4
other_rate      - 0.3
scale_pos_weight - 1.75
max_cat_to_onehot - 22
[Time taken: 11.25s]

-----Cross-validation and prediction-----
Fold #0: F1 = 0.00000
Fold #1: F1 = 0.00000
Fold #2: F1 = 0.00000
Fold #3: F1 = 0.00000
Fold #4: F1 = 0.00000
Avg. F1 = 0.00000 +/- 0.00000
[Time taken: 16.37s]

CPU times: user 40.5 s, sys: 583 ms, total: 41.1 s
Wall time: 27.6 s


In [29]:
%%time
op, tp = run_experiment(
    data=(train[features], train[TARGET], test[features]),
    proba=True,
    n_trials=3
)

[I 2022-11-13 14:12:33,341] A new study created in memory with name: no-name-21e381ed-6be0-4a99-b90d-39e2aa0e2b25


----------Hyperparameter tuning----------


[I 2022-11-13 14:12:37,433] Trial 0 finished with value: 0.06396777611760725 and parameters: {'learning_rate': 0.1, 'reg_alpha': 169.20000000000002, 'reg_lambda': 43.7, 'num_leaves': 315, 'max_depth': 3, 'min_child_samples': 708, 'min_split_gain': 0.14, 'colsample_bytree': 0.55, 'top_rate': 0.4, 'other_rate': 0.3, 'scale_pos_weight': 1.75, 'max_cat_to_onehot': 22}. Best is trial 0 with value: 0.06396777611760725.
[I 2022-11-13 14:12:40,903] Trial 1 finished with value: 0.06396777611760725 and parameters: {'learning_rate': 0.17500000000000002, 'reg_alpha': 23.700000000000003, 'reg_lambda': 133.9, 'num_leaves': 720, 'max_depth': 11, 'min_child_samples': 548, 'min_split_gain': 7.69, 'colsample_bytree': 0.7, 'top_rate': 0.2, 'other_rate': 0.05, 'scale_pos_weight': 11.600000000000001, 'max_cat_to_onehot': 11}. Best is trial 0 with value: 0.06396777611760725.
[I 2022-11-13 14:12:44,365] Trial 2 finished with value: 0.06396777611760725 and parameters: {'learning_rate': 0.225, 'reg_alpha': 183

Best trial: 0 -> Best value: 0.06397
Best hyperparameters:
learning_rate   - 0.1
reg_alpha       - 169.20000000000002
reg_lambda      - 43.7
num_leaves      - 315
max_depth       - 3
min_child_samples - 708
min_split_gain  - 0.14
colsample_bytree - 0.55
top_rate        - 0.4
other_rate      - 0.3
scale_pos_weight - 1.75
max_cat_to_onehot - 22
[Time taken: 11.13s]

-----Cross-validation and prediction-----
Fold #0: F1 = 0.12030
Fold #1: F1 = 0.12030
Fold #2: F1 = 0.12016
Fold #3: F1 = 0.12016
Fold #4: F1 = 0.12031
Avg. F1 = 0.12024 +/- 0.00007
[Time taken: 17.17s]

CPU times: user 41.3 s, sys: 610 ms, total: 41.9 s
Wall time: 28.3 s


In [30]:
optuna.logging.set_verbosity(optuna.logging.ERROR)

# Exp 1

In [31]:
%%time
op1, tp1 = run_experiment(
    data=(train[features], train[TARGET], test[features]),
    proba=False,
    n_trials=200
)

----------Hyperparameter tuning----------
Best trial: 138 -> Best value: 0.14467
Best hyperparameters:
learning_rate   - 0.025
reg_alpha       - 168.3
reg_lambda      - 8.9
num_leaves      - 130
max_depth       - 6
min_child_samples - 306
min_split_gain  - 6.29
colsample_bytree - 0.5
top_rate        - 0.15000000000000002
other_rate      - 0.35000000000000003
scale_pos_weight - 11.950000000000001
max_cat_to_onehot - 3
[Time taken: 1107.12s]

-----Cross-validation and prediction-----
Fold #0: F1 = 0.13963
Fold #1: F1 = 0.13876
Fold #2: F1 = 0.14898
Fold #3: F1 = 0.14815
Fold #4: F1 = 0.14783
Avg. F1 = 0.14467 +/- 0.00450
[Time taken: 15.54s]

CPU times: user 34min 11s, sys: 10.6 s, total: 34min 21s
Wall time: 18min 42s


# Exp 2

In [32]:
%%time
op2, tp2 = run_experiment(
    data=(train[features], train[TARGET], test[features]),
    proba=True,
    n_trials=200
)

----------Hyperparameter tuning----------
Best trial: 162 -> Best value: 0.10325
Best hyperparameters:
learning_rate   - 0.17500000000000002
reg_alpha       - 4.3
reg_lambda      - 50.400000000000006
num_leaves      - 1190
max_depth       - 13
min_child_samples - 56
min_split_gain  - 0.02
colsample_bytree - 0.75
top_rate        - 0.2
other_rate      - 0.5
scale_pos_weight - 1.3
max_cat_to_onehot - 11
[Time taken: 1631.64s]

-----Cross-validation and prediction-----
Fold #0: F1 = 0.17407
Fold #1: F1 = 0.17335
Fold #2: F1 = 0.16927
Fold #3: F1 = 0.17376
Fold #4: F1 = 0.16729
Avg. F1 = 0.17155 +/- 0.00275
[Time taken: 34.61s]

CPU times: user 51min 37s, sys: 17.2 s, total: 51min 54s
Wall time: 27min 46s


# Submission files

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [34]:
NOTEBOOK = '02'
SUBMISSION_PATH = f'/content/drive/MyDrive/data_science_competitions/analytics_vidhya/dataverse_hack/submissions/nb_{NOTEBOOK}'
if not os.path.isdir(SUBMISSION_PATH):
    os.makedirs(SUBMISSION_PATH)

In [35]:
def create_submission_files(test_preds: pd.DataFrame, expt_num: int):
    for col in (test_preds.columns):
        sub = sample_sub.copy()
        sub[TARGET] = test_preds[col]
        sub.to_csv(f'{SUBMISSION_PATH}/{expt_num}_{col}.csv', index=False)

In [36]:
create_submission_files(tp1, '01')
create_submission_files(tp2, '02')